In [1]:
import os
import cv2
import random
import numpy as np
from tkinter import messagebox
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from tkinter.messagebox import showinfo, askquestion

In [3]:
pthim = r'\\10.99.68.52\Kiemendata\Valentina Matos\Dashboard Project\coda python test files\lung 5x'
numims = 2

CT0 = 205
CTA = CT0 + 5
CTC = CT0 - 5
szz = 600

# Choose images to load
imlist = os.listdir(pthim)
numims = min(len(imlist), numims)
imlist = random.sample(imlist, numims)
print(f'Evaluating {numims} randomly selected images to choose a good whitespace detection...')

# Check if the optimal cutoff has already been chosen
outpth = os.path.join(pthim, 'TA')
if not os.path.exists(outpth):
    os.makedirs(outpth)
if os.path.exists(os.path.join(outpth, 'TA_cutoff.mat')):
    print('  Optimal cutoff already chosen, skip this step')

Evaluating 2 randomly selected images to choose a good whitespace detection...


In [ ]:
cts = np.zeros(len(imlist))
for b, nm in enumerate(imlist, 1):

     # Load image
    print(f'  Loading image {b} of {numims}: {nm}')
    im0 = cv2.imread(os.path.join(pthim, nm))
    print('    Image loaded')

    # Crop a region from the image
    do_again = True
    while do_again:
        # Display image
        messagebox.showinfo("Info", "An image will appear\n\nClick on a location at the edge of tissue and whitespace")
        # Display image if dimensions are valid
        if im0.shape[0] > 0 and im0.shape[1] > 0:
            cv2.imshow("Image", im0)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("Invalid image dimensions. Cannot display.")
        x, y = cv2.selectROI("Select ROI", im0)
        x, y, w, h = int(x), int(y), int(w), int(h)
        xx = np.intersect1d(np.arange(im0.shape[1]), np.arange(x - szz, x + szz))
        yy = np.intersect1d(np.arange(im0.shape[0]), np.arange(y - szz, y + szz))

        # plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
        # plt.title('Click on a location at the edge of tissue and whitespace')
        # plt.axis('off')
        # coords = plt.ginput(1)  # Get coordinates from user input
        # plt.close()
        #
        # x, y = int(coords[0][0]), int(coords[0][1])
        #
        # xx = np.intersect1d(np.arange(im0.shape[1]), np.arange(x - szz, x + szz))
        # yy = np.intersect1d(np.arange(im0.shape[0]), np.arange(y - szz, y + szz))


        # Determine if the cropped region is good or not
        im = im0[yy[0]:yy[-1], xx[0]:xx[-1]]
        plt.imshow('Cropped Image', im)
        response = askquestion("Evaluation", "Is this a good location to evaluate tissue and whitespace detection?")
        if response == 'yes':
            do_again = False

    found_CT0 = False
    while not found_CT0:
        img = im[:, :, 1]
        imA = img > CTA
        imA = cv2.morphologyEx(imA.astype(np.uint8), cv2.MORPH_OPEN, np.ones((10, 10)))
        imA = ~cv2.morphologyEx(~imA, cv2.MORPH_OPEN, np.ones((10, 10)))
        imB = img > CT0
        imB = cv2.morphologyEx(imB.astype(np.uint8), cv2.MORPH_OPEN, np.ones((10, 10)))
        imB = ~cv2.morphologyEx(~imB, cv2.MORPH_OPEN, np.ones((10, 10)))
        imC = img > CTC
        imC = cv2.morphologyEx(imC.astype(np.uint8), cv2.MORPH_OPEN, np.ones((10, 10)))
        imC = ~cv2.morphologyEx(~imC, cv2.MORPH_OPEN, np.ones((10, 10)))

        plt.figure(figsize=(10, 5))

        plt.subplot(2, 2, 1)
        plt.imshow(im, cmap='gray')
        plt.title('H&E Image')

        plt.subplot(2, 2, 2)
        plt.imshow(imA, cmap='gray')
        plt.title('IMAGE A: keep more tissue')

        plt.subplot(2, 2, 3)
        plt.imshow(imB, cmap='gray')
        plt.title('IMAGE B: medium')

        plt.subplot(2, 2, 4)
        plt.imshow(imC, cmap='gray')
        plt.title('IMAGE C: keep more whitespace')

        plt.tight_layout()
        plt.show()

        messagebox.showinfo("Note", "Zoom in to evaluate the potential whitespace detection images. "
                                    "Determine which image is best, then press spacebar to continue. "
                                    "In the images, black pixels are tissue, white pixels are whitespace.")
#         response = askquestion("Decision", "Does one of the images look good?")
#         if response == 'yes':
#             found_CT0 = True
#         elif response == 'no':
#             response = askquestion("Decision", "No, keep more tissue or keep more whitespace?")
#             if response == 'yes':
#                 CT0 += 10
#                 CTA = CT0 + 5
#                 CTC = CT0 - 5
#             elif response == 'no':
#                 CT0 -= 10
#                 CTA = CT0 + 5
#                 CTC = CT0 - 5
#
# response = askquestion("Decision", "Which image is best?")
# if response == 'yes':
#     response = 'IMAGE B'
# print(f'    Chose {response}')
# if response == 'IMAGE A':
#     cts[b - 1] = CTA
# elif response == 'IMAGE B':
#     cts[b - 1] = CT0
# elif response == 'IMAGE C':
#     cts[b - 1] = CTC
#
# np.save(os.path.join(outpth, 'TA_cutoff.npy'), [imlist, cts])

Evaluating 2 randomly selected images to choose a good whitespace detection...
  Loading image 1 of 2: TVWT 4-1 1.tif
    Image loaded


In [6]:
import matplotlib.pyplot as plt
import numpy as np

# Some example data to display
x = np.linspace(0, 2 * np.pi, 400)
y = np.sin(x ** 2)

fig, axs = plt.subplots(2, 2)
axs[0, 0].plot(x, y)
axs[0, 0].set_title("main")
axs[1, 0].plot(x, y**2)
axs[1, 0].set_title("shares x with main")
axs[1, 0].sharex(axs[0, 0])
axs[0, 1].plot(x + 1, y + 1)
axs[0, 1].set_title("unrelated")
axs[1, 1].plot(x + 2, y + 2)
axs[1, 1].set_title("also unrelated")
fig.tight_layout()
plt.show()


In [20]:
def determine_optimal_TA(pthim=None, numims=None):
    if pthim is None or numims is None:
        pthim = r'\\10.99.68.52\Kiemendata\Valentina Matos\tissues for methods paper\human skin\10x'
        numims = 5

    CT0 = 205
    CTA = CT0 + 5
    CTC = CT0 - 5
    szz = 600

    # Choose images to load
    imlist = os.listdir(pthim)
    numims = min(len(imlist), numims)
    imlist = random.sample(imlist, numims)
    print(f'Evaluating {numims} randomly selected images to choose a good whitespace detection...')

    # Check if the optimal cutoff has already been chosen
    outpth = os.path.join(pthim, 'TA')
    if not os.path.exists(outpth):
        os.makedirs(outpth)
    if os.path.exists(os.path.join(outpth, 'TA_cutoff.mat')):
        print('  Optimal cutoff already chosen, skip this step')
        return

    cts = np.zeros(len(imlist))
    for b, nm in enumerate(imlist, 1):
        # Load image
        print(f'  Loading image {b} of {numims}: {nm}')
        im0 = cv2.imread(os.path.join(pthim, nm))
        print('    Image loaded')

        # Crop a region from the image
        do_again = True
        while do_again:
            # Display image
            plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
            plt.title('Click on a location at the edge of tissue and whitespace')
            plt.axis('off')
            coords = plt.ginput(1)  # Get coordinates from user input
            plt.close()

            x, y = int(coords[0][0]), int(coords[0][1])

            xx = np.intersect1d(np.arange(im0.shape[1]), np.arange(x - szz, x + szz))
            yy = np.intersect1d(np.arange(im0.shape[0]), np.arange(y - szz, y + szz))


            # Determine if the cropped region is good or not
            im = im0[yy[0]:yy[-1], xx[0]:xx[-1]]
            cv2.imshow('Cropped Image', im)
            response = askquestion("Evaluation", "Is this a good location to evaluate tissue and whitespace detection?")
            if response == 'yes':
                do_again = False

        found_CT0 = False
        while not found_CT0:
            img = im[:, :, 1]
            imA = img > CTA
            imA = cv2.morphologyEx(imA.astype(np.uint8), cv2.MORPH_OPEN, np.ones((10, 10)))
            imA = ~cv2.morphologyEx(~imA, cv2.MORPH_OPEN, np.ones((10, 10)))
            imB = img > CT0
            imB = cv2.morphologyEx(imB.astype(np.uint8), cv2.MORPH_OPEN, np.ones((10, 10)))
            imB = ~cv2.morphologyEx(~imB, cv2.MORPH_OPEN, np.ones((10, 10)))
            imC = img > CTC
            imC = cv2.morphologyEx(imC.astype(np.uint8), cv2.MORPH_OPEN, np.ones((10, 10)))
            imC = ~cv2.morphologyEx(~imC, cv2.MORPH_OPEN, np.ones((10, 10)))

            cv2.imshow('H&E Image', im)
            cv2.imshow('IMAGE A: keep more tissue', imA)
            cv2.imshow('IMAGE B: medium', imB)
            cv2.imshow('IMAGE C: keep more whitespace', imC)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            messagebox.showinfo("Note", "Zoom in to evaluate the potential whitespace detection images. "
                                        "Determine which image is best, then press spacebar to continue. "
                                        "In the images, black pixels are tissue, white pixels are whitespace.")
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            response = askquestion("Decision", "Does one of the images look good?")
            if response == 'yes':
                found_CT0 = True
            elif response == 'no':
                response = askquestion("Decision", "No, keep more tissue or keep more whitespace?")
                if response == 'yes':
                    CT0 += 10
                    CTA = CT0 + 5
                    CTC = CT0 - 5
                elif response == 'no':
                    CT0 -= 10
                    CTA = CT0 + 5
                    CTC = CT0 - 5

        response = askquestion("Decision", "Which image is best?")
        if response == 'yes':
            response = 'IMAGE B'
        print(f'    Chose {response}')
        if response == 'IMAGE A':
            cts[b - 1] = CTA
        elif response == 'IMAGE B':
            cts[b - 1] = CT0
        elif response == 'IMAGE C':
            cts[b - 1] = CTC

    np.save(os.path.join(outpth, 'TA_cutoff.npy'), [imlist, cts])

In [21]:
if __name__ == "__main__":
    determine_optimal_TA()

Evaluating 5 randomly selected images to choose a good whitespace detection...
  Loading image 1 of 5: OTS-21-11041 - 2021-11-15 09.49.33.tif
    Image loaded


AttributeError: 'NoneType' object has no attribute 'split'

In [6]:
messagebox.showinfo("Note", "Zoom in to evaluate the potential whitespace detection images. "
                                    "Determine which image is best, then press spacebar to continue. "
                                    "In the images, black pixels are tissue, white pixels are whitespace.")

'ok'